[View in Colaboratory](https://colab.research.google.com/github/mlindauer/lab_course_add/blob/master/Run_SMAC_Cheap_function_(Full_CS).ipynb)

# Using SMAC to optimize an artificial black box function (Rosenbrock)

* Optimization of 2d rosenbrock function 
* Using fmin interface and SMAC facade
* Example on how to use the configspace

## Installation of SMAC and its dependencies

In [0]:
!apt-get install swig -y
!pip install Cython
!pip install pyrfr==0.8.0 --no-cache --user
# hack to find pyrfr
import sys
sys.path.insert(0,"./.local/lib/python3.6/site-packages")

!pip install git+https://github.com/automl/SMAC3.git@development
  
import logging
logging.basicConfig(level=logging.INFO)

## Define Function to be optimized
Rosenbrock : https://en.wikipedia.org/wiki/Rosenbrock_function

In [0]:
import math
import numpy as np

def rosenbrock_2d(x):
    x1 = float(x[0])
    x2 = float(x[1])

    val = 100. * (x2 - x1 ** 2.) ** 2. + (1 - x1) ** 2.
    return val

## Optimize with SMAC -- fmin Interface

In [0]:
from smac.facade.func_facade import fmin_smac

MAX_FUN = 200

x, cost, smac = fmin_smac(func=rosenbrock_2d,
                       x0=[-3, -4], # default values
                       bounds=[(-5, 5), (-5, 5)], # bounds of each x
                       maxfun=MAX_FUN, # maximal number of function evaluations 
                       rng=1234 # random seed
                       )

print("Best x: %f,%f" %(x[0], x[1]))
print("Best y: %f" %(cost))

## How has SMAC performed over time?

In [0]:
import matplotlib.pyplot as plt
import numpy as np

# get runhistory, all runs ever evaluated by smac
runhistory = smac.get_runhistory()

# extract x value and corresponding y value
y_smac = []
for entry in runhistory.data: # iterate over data because it is an OrderedDict
  config_id = entry.config_id # look up config id
  config = runhistory.ids_config[config_id] # look up config
  y_ = runhistory.get_cost(config) # get cost
  y_smac.append(y_)
y_smac = np.array(y_smac)


# Plotting
ax = plt.subplot(111)
ax.plot(y_smac, 'o')
plt.xlabel('function evaluation')
plt.ylabel('function value')

# let's plot only the best function value at each time step
y_best = np.zeros(MAX_FUN)
for id in range(MAX_FUN):
  y_best[id] = np.min(y_smac[:id+1])
  
ax.step(y_best, 'k', where="post") # please note that we use step function!
ax.set_yscale("log")

## Optimize with SMAC -- SMAC interface

In [0]:
def rosenbrock_2d(x):
    # use parameter names! 
    x1 = float(x["x1"])
    x2 = float(x["x2"])

    val = 100. * (x2 - x1 ** 2.) ** 2. + (1 - x1) ** 2.
    return val

In [0]:
from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, \
    UniformFloatHyperparameter

# Build configuration space

cs = ConfigurationSpace(seed=1234)
# for illustration, let's make the first parameter categorical
x1 = CategoricalHyperparameter(name="x1", choices=range(-5,6), default_value=-3)
# the second will be a continuous parameter
x2 = UniformFloatHyperparameter(name="x2", lower=-5,upper=5, default_value=-4)

# add parameters to config space
cs.add_hyperparameters([x1, x2])

print(cs)


In [0]:
from smac.scenario.scenario import Scenario
from smac.facade.smac_facade import SMAC

# Build an scenario
scenario = Scenario({"run_obj": "quality",  # we optimize quality 
                     "runcount-limit": MAX_FUN,  # maximum number of function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })

# start SMAC
smac = SMAC(scenario=scenario, 
            rng=np.random.RandomState(1234),
            tae_runner=rosenbrock_2d)

best_x = smac.optimize()

cost = smac.get_tae_runner().run(best_x, 1)[1]
print("Best x: %f,%f" %(best_x["x1"], best_x["x2"]))
print("Best y: %f" %(cost))